<h1>Using User-Based Collabortive Filtering to Predict a User's Next Foursquare Check-In to a New Location</h1>
<h2>MS&E 234</h2>

<h3>Setup</h3>

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

Note: The Check-In dataset below was already processed from the 'Global-scale Check-in Dataset with User Social Networks', in order to 1) only consider data points of users who we know the gender of, 2) identify the category of venues using another dataset.

In [92]:
# Check-In Data
check_ins = pd.read_csv('gender-checkins-complete.csv', delimiter='\t', index_col=0)

In [143]:
# check user with id 541
check_ins[(check_ins.User_ID == 54) & (check_ins.Venue_ID == '4de0117c45dd3eae8764d6ac')]

,User_ID,Venue_ID,UTC_Time,Timezone_Offset,Lat,Long,Category,Country_Code
28025,54,4de0117c45dd3eae8764d6ac,Fri Apr 13 22:48:37 +0000 2012,-420,37.781213,-122.402973,Tech Startup,US
38400,54,4de0117c45dd3eae8764d6ac,Mon Apr 16 18:25:45 +0000 2012,-420,37.781213,-122.402973,Tech Startup,US
88863,54,4de0117c45dd3eae8764d6ac,Thu May 03 19:07:58 +0000 2012,-420,37.781213,-122.402973,Tech Startup,US
105646,54,4de0117c45dd3eae8764d6ac,Tue May 08 16:12:45 +0000 2012,-420,37.781213,-122.402973,Tech Startup,US
151204,54,4de0117c45dd3eae8764d6ac,Tue May 22 15:32:18 +0000 2012,-420,37.781213,-122.402973,Tech Startup,US
186459,54,4de0117c45dd3eae8764d6ac,Thu Jun 07 03:37:30 +0000 2012,-420,37.781213,-122.402973,Tech Startup,US
284996,54,4de0117c45dd3eae8764d6ac,Thu Aug 09 17:33:19 +0000 2012,-420,37.781213,-122.402973,Tech Startup,US
312667,54,4de0117c45dd3eae8764d6ac,Fri Sep 14 20:59:39 +0000 2012,-420,37.781213,-122.402973,Tech Startup,US
324125,54,4de0117c45dd3eae8764d6ac,Tue Oct 16 15:43:10 +0000 2012,-420,37.781213,-122.402973,Tech Startup,US
348655,54,4de0117c45dd3eae8764d6ac,Thu Nov 01 18:01:51 +0000 2012,-420,37.781213,-122.402973,Tech Startup,US


In [3]:
check_ins.head()

,User_ID,Venue_ID,UTC_Time,Timezone_Offset,Lat,Long,Category,Country_Code
0,21939,4dd53b151f6ec4e0bb8c0480,Tue Apr 03 18:00:49 +0000 2012,-240,39.285600,-76.612047,Clothing Store,US
1,163646,4b70040ff964a52080032de3,Tue Apr 03 18:01:31 +0000 2012,-240,25.716845,-80.281378,College Cafeteria,US
2,256534,4b29929cf964a5200fa124e3,Tue Apr 03 18:01:37 +0000 2012,-360,40.726135,-111.852087,American Restaurant,US
3,176836,4b66f88ff964a520eb322be3,Tue Apr 03 18:01:40 +0000 2012,-300,29.661129,-95.115077,Community College,US
4,181560,4bc7086715a7ef3bef9878da,Tue Apr 03 18:02:41 +0000 2012,-240,40.745164,-73.982519,Medical Center,US


### Preprocessing

The following functions reduce the data to users who have made 80+ check-ins in the past.

In [4]:
threshold = 80 # Any user that has less than (threshold) check-ins will be removed.

value_counts = check_ins['User_ID'].value_counts() # Specific column 
to_remove = value_counts[value_counts <= threshold].index
check_ins = check_ins[~check_ins['User_ID'].isin(to_remove)]

In [5]:
# threshold = 2000 # Any user that has more than (threshold) check-ins will be removed.

# value_counts = check_ins[0].value_counts() # Specific column 
# to_remove = value_counts[value_counts >= threshold].index
# check_ins = check_ins[~check_ins[0].isin(to_remove)]

In [6]:
"There are %d check-ins, made by %d users, in %d locations." %(len(check_ins), check_ins['User_ID'].nunique(), check_ins['Venue_ID'].nunique())

'There are 465613 check-ins, made by 2290 users, in 184838 locations.'

In [7]:
#split into train/test
split = 0.8

X_train = (check_ins.groupby('User_ID',group_keys=False)
        .apply(lambda x: x.nlargest(int(len(x) * split), 'User_ID')))

In [8]:
X_test = check_ins[~check_ins.isin(X_train)].dropna()

In [9]:
del check_ins

In [10]:
X_train_counts = X_train.groupby(['User_ID','Venue_ID']).size().reset_index(name="Frequency")

In [11]:
Total_Visits = X_train.groupby(['User_ID']).size().reset_index(name="Total_Visits")
X_train_counts = pd.merge(X_train_counts, Total_Visits, on = 'User_ID', how='left', sort = 'False')
del X_train

In [12]:
"The users have %d check-ins on average in the training set." %(np.mean(Total_Visits['Total_Visits']))

'The users have 162 check-ins on average in the training set.'

In [13]:
X_train_counts['Adj_Freq'] = X_train_counts['Frequency'] / X_train_counts['Total_Visits'] * 1.0

In [14]:
X_train_counts.head()

,User_ID,Venue_ID,Frequency,Total_Visits,Adj_Freq
0,54,3fd66200f964a5204ded1ee3,1,108,0.009259
1,54,3fd66200f964a5209fe61ee3,1,108,0.009259
2,54,40919700f964a520e1f21ee3,1,108,0.009259
3,54,409ad180f964a520eef21ee3,2,108,0.018519
4,54,41059b00f964a520850b1fe3,1,108,0.009259


In [15]:
X_train_counts = X_train_counts.pivot(index='User_ID', columns='Venue_ID', values='Adj_Freq')

In [16]:
#X_train_counts = X_train_counts.fillna(0)

In [17]:
X_train_counts.head()

Venue_ID,3fd66200f964a52000e71ee3,3fd66200f964a52000ee1ee3,3fd66200f964a52000f11ee3,3fd66200f964a52001e81ee3,3fd66200f964a52002f01ee3,3fd66200f964a52003e71ee3,3fd66200f964a52003e81ee3,3fd66200f964a52004e61ee3,3fd66200f964a52005e71ee3,3fd66200f964a52005eb1ee3,...,523dd9e7498e092f9a033bb5,52458a1411d2b5993ac0425b,524842f711d2289df8ba9723,524a968f11d2921536d1e87d,524dd87611d2233c88389112,5255b0cb498e1ef25a04735c,5261c77611d2d6de373293f2,526f51e7498e26445f274a5e,528218eb498e0679e73b2490,529d18fb11d26ddfd0f8152d
User_ID,,,,,,,,,,,,,,,,,,,,,
54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
182,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
346,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.005263,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
419,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.014706,NaN,NaN,NaN,NaN,NaN,NaN
541,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
sim_scores = cosine_similarity(X_train_counts)

In [ ]:
#use nearest 3 neighbors
sim_scores.shape

In [ ]:
sim_scores

FOR EACH USER (X):

1- Use the cosine similarity matrix above to get top 5 most similar users to (X)

2- Refer to top 5 users similar to the user (X) as: u1, u2, u3, u4, u5

3- For every location L1, L2, L3.., calculate the probabilty of user X visiting location L as = (L*sim(u1) + L*sim(u2) ... + L*sim(u5))/ (sim(u1)+sim(u2)...+sim(u5))

4- Sort probabilities from high to low, use top 1 as prediction

In [ ]:
# def matrix_factorization(R, P, Q, K, steps=5000, alpha=0.0002, beta=0.02):
#     Q = Q.T
#     for step in range(steps):
#         for i in range(len(R)):
#             for j in range(len(R[i])):
#                 if R[i][j] > 0:
#                     eij = R[i][j] - numpy.dot(P[i,:],Q[:,j])
#                     for k in range(K):
#                         P[i][k] = P[i][k] + alpha * (2 * eij * Q[k][j] - beta * P[i][k])
#                         Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k] - beta * Q[k][j])
#         eR = numpy.dot(P,Q)
#         e = 0
#         for i in range(len(R)):
#             for j in range(len(R[i])):
#                 if R[i][j] > 0:
#                     e = e + pow(R[i][j] - numpy.dot(P[i,:],Q[:,j]), 2)
#                     for k in range(K):
#                         e = e + (beta/2) * (pow(P[i][k],2) + pow(Q[k][j],2))
#         if e < 0.001:
#             break
#     return P, Q.T

In [ ]:
# R = X_train_counts.values

# R = numpy.array(R)

# N = len(R)
# M = len(R[0])
# K = 2

# P = numpy.random.rand(N,K)
# Q = numpy.random.rand(M,K)

# nP, nQ = matrix_factorization(R, P, Q, K)
# nR = numpy.dot(nP, nQ.T)

In [18]:
df_nans_spcoo = X_train_counts.to_sparse()

In [19]:
df_nans_spcoo = df_nans_spcoo.to_coo()

In [22]:
class MaxNormMF():
    def __init__(self, coo_mat):
        self.M, self.N = coo_mat.shape
        self.NNZ = coo_mat.nnz
        self.ratings = coo_mat.data
        self.rows = coo_mat.row
        self.cols = coo_mat.col

    def calc_train_loss(self):
        n = self.NNZ
        e = 0.0
        for ind in range(n):
            i, j = self.rows[ind], self.cols[ind]
            r = self.ratings[ind]
            e += math.pow(np.dot(self.L[i, :], self.R[j, :].T) - r, 2)
        return math.pow(e / n, 0.5)

    def train(self, k=10, b=2, epochs=50, gamma=0.1, gamma_red_factor=0.9,
              show_progress=True, show_train_error=True):
        self.start_time = time.perf_counter()

        self.k = k
        self.b = b
        self.L = np.random.randn(len(self.rows), self.k) * 0.001
        self.R = np.random.randn(len(self.cols), self.k) * 0.001

        b_sqrt = math.sqrt(self.b)
        def project(v):
            v_norm = np.linalg.norm(v)
            if np.square(v_norm) >= self.b:
                return (b_sqrt * v) / v_norm
            else:
                return v

        perm_indices = np.arange(self.NNZ)

        for epoch in range(epochs):
            if show_progress:
                if epoch % 5 == 0:
                    print('epoch: ', epoch)
                    print(' secs: ', time.perf_counter() - self.start_time)
            if show_train_error:
                if (epoch != 0) and (epoch % 5 == 0):
                    e = self.calc_train_loss()
                    print(' -> ', e)

            np.random.shuffle(perm_indices)

            for ind in perm_indices:
                i, j = self.rows[ind], self.cols[ind]
                r = self.ratings[ind]

                pred = np.dot(self.L[i, :], self.R[j, :].T)

                # update rule
                grad = pred - r
                L_pre = self.L[i, :] - gamma * grad * self.R[j, :]
                R_pre = self.R[j, :] - gamma * grad * self.L[i, :]

                # projection
                self.L[i, :] = project(L_pre)
                self.R[j, :] = project(R_pre)

            # decr learning-rate
            gamma *= gamma_red_factor

        print(self.L)
        print(self.R)

    def predict(self, u, i):
        return np.dot(self.L[u, :], self.R[i, :].T)

In [23]:
maxnorm = MaxNormMF(df_nans_spcoo)
maxnorm.train(k=11, b=1.35, epochs=350, gamma=0.5, gamma_red_factor=0.99)

epoch:  0
 secs:  0.20742157300003328
epoch:  5
 secs:  32.38062220900008
 ->  0.02865723403475664
epoch:  10
 secs:  69.29154716100004
 ->  0.028642038006389148
epoch:  15
 secs:  103.907159138
 ->  0.028462699210801797
epoch:  20
 secs:  137.57626221600003
 ->  0.027696076475011294
epoch:  25
 secs:  170.25569236700005
 ->  0.026372661951767653
epoch:  30
 secs:  203.22392342299997
 ->  0.02453263353834855
epoch:  35
 secs:  236.113140198
 ->  0.022342297891839304
epoch:  40
 secs:  269.25997461200006
 ->  0.020137570082691257
epoch:  45
 secs:  302.30911705000005
 ->  0.01816509658611381
epoch:  50
 secs:  336.4913802120001
 ->  0.016423721309759373
epoch:  55
 secs:  369.4809643520001
 ->  0.014875316393494072
epoch:  60
 secs:  403.75953645200013
 ->  0.013492686862604032
epoch:  65
 secs:  437.4204370420001
 ->  0.012264493563576022
epoch:  70
 secs:  471.8402848930001
 ->  0.01117725015701221
epoch:  75
 secs:  504.67416659
 ->  0.01021229082216411
epoch:  80
 secs:  537.5798137

In [24]:
shifted_mask = np.isnan(X_train_counts.values)
shifted_mask

array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]])

In [98]:
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import Imputer

nan_indices = shifted_mask.nonzero()
n_nans = len(nan_indices[0])
df_nans_spcoo_dok = df_nans_spcoo.todok()  # needed for single-item-based access!
for i in range(n_nans):
    if row == 10:
        break
    row, col = nan_indices[0][i], nan_indices[1][i]
    prediction = maxnorm.predict(row, col)
    df_nans_spcoo_dok[row, col] = prediction
    #print('row,col,pred: ', row, col, prediction)

df_nans_spcoo_imputed = df_nans_spcoo_dok.tocoo()

""" Compare: incomplete re-mapping!!! Just a demo """

print(df_nans_spcoo_imputed.todense()[:, :])

[[ 0.00933319  0.00438639  0.0006445  ... -0.00037692 -0.00030714
  -0.0004148 ]
 [-0.00043191 -0.00077543 -0.00142018 ...  0.00225391  0.00101829
   0.00216516]
 [-0.0128044  -0.00123569 -0.0043917  ... -0.00193585 -0.00302979
  -0.00518675]
 ...
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]]


In [137]:
np.sort(df_nans_spcoo_imputed.todense()[9])

matrix([[-0.22173739, -0.20142573, -0.18718202, ...,  0.18896342,
          0.21580071,  0.23316328]])

In [138]:
np.argsort(df_nans_spcoo_imputed.todense()[9])

matrix([[118530, 142412,  99193, ..., 104540, 104973,  16542]])

In [101]:
X_train_counts.head(10)

Venue_ID,3fd66200f964a52000e71ee3,3fd66200f964a52000ee1ee3,3fd66200f964a52000f11ee3,3fd66200f964a52001e81ee3,3fd66200f964a52002f01ee3,3fd66200f964a52003e71ee3,3fd66200f964a52003e81ee3,3fd66200f964a52004e61ee3,3fd66200f964a52005e71ee3,3fd66200f964a52005eb1ee3,...,523dd9e7498e092f9a033bb5,52458a1411d2b5993ac0425b,524842f711d2289df8ba9723,524a968f11d2921536d1e87d,524dd87611d2233c88389112,5255b0cb498e1ef25a04735c,5261c77611d2d6de373293f2,526f51e7498e26445f274a5e,528218eb498e0679e73b2490,529d18fb11d26ddfd0f8152d
User_ID,,,,,,,,,,,,,,,,,,,,,
54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
182,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
346,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.005263,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
419,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.014706,NaN,NaN,NaN,NaN,NaN,NaN
541,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
545,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
562,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
799,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
884,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [155]:
colname = X_train_counts.columns[2175]
colname

'440da2cbf964a52091301fe3'

In [160]:
colname = X_train_counts.columns[2175]
X_test[(X_test.User_ID == 150255) & (X_test.Venue_ID == '4b35e986f964a5205a2f25e3')]

,User_ID,Venue_ID,UTC_Time,Timezone_Offset,Lat,Long,Category,Country_Code
624080,150255.0,4b35e986f964a5205a2f25e3,Fri Jun 07 11:24:43 +0000 2013,600.0,-37.868311,144.986798,Gay Bar,AU
